In [1]:
import cobra

In [2]:
import pandas as pd

In [3]:
from cobra.io import load_model

In [4]:
from cobra.flux_analysis import flux_variability_analysis

In [84]:
model = load_model("iJO1366") #no need to import from matlab

In [85]:
biomass = "BIOMASS_Ec_iJO1366_core_53p95M"

In [112]:
model.objective = biomass

In [113]:
maxG = model.optimize().objective_value

In [114]:
maxG

0.9823718127269768

What I should change is the init flux of WT

In [115]:
model.reactions.get_by_id("O2tex").summary().to_frame().loc["O2tex", "flux"]

17.57893353025448

In [116]:
initFluxList = []
for rec in optList:
    initFlux = model.reactions.get_by_id(rec).summary().to_frame().loc[rec, "flux"]
    initFluxList.append(initFlux)

In [117]:
initFluxList

[-0.24776399488787101,
 -0.442588955159698,
 0.0,
 0.0,
 -0.9476263040826025,
 -0.442588955159698,
 -0.858857222341002,
 -0.28540750037975604,
 0.9476263040826025,
 -17.57893353025448]

In [118]:
init_df = pd.DataFrame()

In [119]:
init_df["rec"] = optList
init_df["initFlux"] = initFluxList

In [120]:
init_df

,rec,initFlux
0,EX_so4_e,-0.247764
1,IPPMIb,-0.442589
2,SUCCtex,0.000000
3,EX_ac_e,0.000000
4,EX_pi_e,-0.947626
5,IPPMIa,-0.442589
6,KARA1,-0.858857
7,ILETA,-0.285408
8,PItex,0.947626
9,EX_o2_e,-17.578934


In [121]:
model.reactions.get_by_id(biomass).lower_bound = maxG*0.5 #min to half of max growth rate

In [122]:
model.objective = 'EX_succ_e'

In [124]:
flux_variability_analysis(model, "EX_succ_e", loopless=True)

,minimum,maximum
EX_succ_e,8.56108,8.56108


In [125]:
optFluxList = []
for rec in optList:
    optFlux = model.reactions.get_by_id(rec).summary().to_frame().loc[rec, "flux"]
    optFluxList.append(optFlux)

In [126]:
init_df["optFlux"] = optFluxList

In [127]:
init_df

,rec,initFlux,optFlux
0,EX_so4_e,-0.247764,-0.123882
1,IPPMIb,-0.442589,-0.221294
2,SUCCtex,0.000000,-8.561080
3,EX_ac_e,0.000000,0.000000
4,EX_pi_e,-0.947626,-0.473813
5,IPPMIa,-0.442589,-0.221294
6,KARA1,-0.858857,-0.429429
7,ILETA,-0.285408,-0.142704
8,PItex,0.947626,0.473813
9,EX_o2_e,-17.578934,-8.825687


The max and min of Foce set should be the WT

In [107]:
opt_df = pd.read_csv("OptForce_Ecoli_Suc_Comb6_6_filter.csv")

In [108]:
opt_dict = dict(zip(opt_df["Force Set"], opt_df["Type of regulation"]))

In [111]:
opt_dict

{'EX_so4_e': 'upregulation',
 'IPPMIb': 'upregulation',
 'SUCCtex': 'downregulation',
 'EX_ac_e': 'knockout',
 'EX_pi_e': 'knockout',
 'IPPMIa': 'upregulation',
 'KARA1': 'upregulation',
 'ILETA': 'upregulation',
 'PItex': 'knockout',
 'EX_o2_e': 'knockout'}

In [109]:
optList = list(opt_df["Force Set"])

In [110]:
opt_df

,Set number,Force Set,Type of regulation,Min flux in Wild Type (mmol/gDW hr),Max flux in Wild Type (mmol/gDW hr),Min flux in Mutant (mmol/gDW hr),Max flux in Mutant (mmol/gDW hr),Achieved flux (mmol/gDW hr),Objective function (mmol/gDW hr),Minimum guaranteed for target (mmol/gDW hr),Maximum guaranteed for target (mmol/gDW hr),Maximum growth rate (1/hr)
0,1,EX_so4_e,upregulation,-55.200639,-2.269890,0,0,0,400.0,400.0,400.0,0.0
1,1,IPPMIb,upregulation,-6.847363,-4.054779,0,0,0,NaN,NaN,NaN,NaN
2,1,SUCCtex,downregulation,-211.619609,0.000000,-400,-400,-400,NaN,NaN,NaN,NaN
3,1,EX_ac_e,knockout,0.000000,432.752266,0,0,0,NaN,NaN,NaN,NaN
4,1,EX_pi_e,knockout,-33.492968,-8.681679,0,0,0,NaN,NaN,NaN,NaN
5,2,IPPMIa,upregulation,-6.847363,-4.054779,0,0,0,400.0,400.0,400.0,0.0
6,2,KARA1,upregulation,-10.661005,-7.868421,0,0,0,NaN,NaN,NaN,NaN
7,2,ILETA,upregulation,-5.407345,-2.614761,0,0,0,NaN,NaN,NaN,NaN
8,2,PItex,knockout,8.681679,33.492968,0,0,0,NaN,NaN,NaN,NaN
9,3,EX_o2_e,knockout,-100.000000,-51.223218,-100,0,0,NaN,NaN,NaN,NaN


In [142]:
obj = "EX_succ_e"

In [143]:
def getFVA(model, combList, combDict, obj):
    #fva_df = pd.DataFrame(columns=['minimum', 'maximum'])
    with model:
        try:
            for rec in combList:
                if opt_dict[rec] == 'downregulation':
                    l_bound, u_bound = setBounds(ori_b_df, rec, "downregulation")
                    model.reactions.get_by_id(rec).lower_bound = l_bound
                    model.reactions.get_by_id(rec).upper_bound = u_bound
                elif opt_dict[rec] == 'upregulation':
                    l_bound, u_bound = setBounds(ori_b_df, rec, "upregulation")
                    model.reactions.get_by_id(rec).lower_bound = l_bound
                    model.reactions.get_by_id(rec).upper_bound = u_bound
                elif opt_dict[rec] == 'knockout':
                    model.reactions.get_by_id(rec).knock_out()
            temp_df = flux_variability_analysis(model, obj)
            #fva_df = pd.concat([fva_df, temp_df], join="inner")
        except:
            temp_df = pd.DataFrame(columns=['minimum', 'maximum'])
            temp_df.loc[obj] = [0, 0]
            #fva_df = pd.concat([fva_df, temp_df], join="inner")
    return temp_df

In [144]:
getFVA(model, optList[:3], opt_dict, obj)

,minimum,maximum
EX_succ_e,0,0


In [145]:
flux_variability_analysis(model, obj)

,minimum,maximum
EX_succ_e,8.56108,8.56108


In [148]:
model.reactions.get_by_id("EX_pi_e").knock_out()

In [149]:
flux_variability_analysis(model, obj)

Infeasible: There is no optimal solution for the chosen objective! (infeasible).

In [135]:
init_df = init_df.set_index("rec")

In [139]:
def setBounds(ori_b_df, rec ,regulation):
    if regulation == "downregulation":
        if ori_b_df.loc[rec]["initFlux"] < 0:
            l_bound = -1000
            u_bound = ori_b_df.loc[rec]["initFlux"] * 2
        elif ori_b_df.loc[rec]["initFlux"] == 0:
            l_bound = -1000
            u_bound = -8
        else:
            l_bound = -1000
            u_bound = ori_b_df.loc[rec]["initFlux"] * 0.5
    elif regulation == "upregulation":
        if ori_b_df.loc[rec]["initFlux"] <= 0:
            l_bound = -1000
            u_bound = ori_b_df.loc[rec]["initFlux"] * 0.5
        elif ori_b_df.loc[rec]["initFlux"] == 0:
            l_bound = -1000
            u_bound = 8
        else:
            l_bound = ori_b_df.loc[rec]["initFlux"] * 2
            u_bound = 1000
    return l_bound, u_bound

In [140]:
setBounds(init_df, "SUCCtex", "downregulation")

(-1000, -8)

In [141]:
setBounds(init_df, "EX_so4_e", "upregulation")

(-1000, -0.12388199744393551)